In [ ]:
import numpy as np
import csv,random
from __future__ import division
import matplotlib.pyplot as plt
import scipy as sp
import scipy.stats
from sklearn.metrics import f1_score
from sklearn.metrics import matthews_corrcoef
from sklearn.metrics import confusion_matrix
from numpy.random import uniform
#load all the functions
%run  myfunctions.ipynb

In [ ]:
def sur_error_rate(preds_matrix, P0=0.51, kappa=0.02):
#compute the error rates
#multi-testing for selecting the right solution for error rates

    label_matrix = np.zeros_like(preds_matrix, dtype=int)
    label_matrix[preds_matrix>=0.5] = 1
    label_matrix[preds_matrix<0.5] = 2
    label_matrix[preds_matrix<0] = -1
    
    #compute the error rates
    #multi-testing for selecting the right solution for error rates

    num_q = int(len(label_matrix)/1)
    num_w = len(label_matrix[0])
    print(num_q,num_w)

    num_hypo = 11
    cnt_l = 0
    cnt_h = 0
    p0_l = []
    p1_l = []
    P0_est_l = []
    P1_post_l = []
    p0_h = []
    p1_h = []
    P0_est_h = []
    P1_post_h = []

    for t in range(num_hypo):
        kk = int(np.ceil(num_w*2))
        train_data = []
        train_data_Y = []
        for q in range(int(num_q/1)):
            ans_temp = label_matrix[q][:]
            list_taken = [] # extract the list of valid answers. 
            for ii in range(len(ans_temp)):
                if float(ans_temp[ii]) > 0: # -1 is regarded as not taking the task
                    list_taken.append(ii)
            #print list_taken
            
            data_temp = [float(ans_temp[x])-1 for x in list_taken]
            for k in range(kk):
                if len(data_temp)<3: continue
                data_temp2 = random.sample(data_temp,3)
                train_data.append(data_temp2)
        
        p0,p1,P0_est,ind = error_rate_priorfree(P0,train_data)
        print(p0,p1,ind)
        if (max(p0,p1) <= 1) & (min(p0,p1) > 0):
            if ind == 1:
                cnt_l = cnt_l + 1
                p0_l.append(p0)
                p1_l.append(p1)
                P0_est_l.append(P0_est)
                #P1_post_l.append(P1_post)
            else:
                cnt_h = cnt_h + 1
                p0_h.append(p0)
                p1_h.append(p1)
                P0_est_h.append(P0_est)
                #P1_post_h.append(P1_post)

    print(np.mean(p0_l),np.mean(p1_l))
    print(np.mean(p0_h),np.mean(p1_h))

    if cnt_l >= cnt_h:
        p0 = np.mean(p0_l)
        p1 = np.mean(p1_l)
        P0_est = np.mean(P0_est_l)
        #P1_post = np.mean(P1_post_l)
    else:
        p0 = np.mean(p0_h)
        p1 = np.mean(p1_h)
        P0_est = np.mean(P0_est_h)
        #P1_post = np.mean(P1_post_h)

        
    print('Estimated e0,e1,P0:~',p0,p1,P0_est)#,P1_post
    if np.isnan(p0) or np.isnan(p1) or np.isnan(P0_est):
        return np.NAN, np.NAN
    
    ###count in individual error rates
    err_ind = []
    for ww in range(num_w):
        if ww%20==0: print(ww)
        #kk = int(np.ceil(num_w*5))
        kk = 2000
        train_data = []
        ind_predict = []
        q_list = []
        for q in range(int(num_q/1)):
            if float(label_matrix[q][ww]) > 0:
                q_list.append(q)
        
        for q in q_list:
            ans_temp = label_matrix[q][:]
            list_taken = [] # extract the list of valid answers. 
            for ii in range(len(ans_temp)):
                if float(ans_temp[ii]) > 0: # -1 is regarded as not taking the task
                    list_taken.append(ii)
            #print list_taken
            
            data_temp = [float(ans_temp[x])-1 for x in list_taken]
            for k in range(kk):
                ind_predict.append(float(label_matrix[q][ww])-1)
                data_temp2 = random.sample(data_temp,1)
                train_data.append(data_temp2[0])
        

        p0_m,p1_m = machine_error(P0_est,p0,p1,train_data,ind_predict)
        if abs(p0_m) <= 0.05:
            p0_m = 0
        if abs(p1_m) <= 0.05:
            p1_m = 0
            
        if abs(p0_m-1) <= 0.05:
            p0_m = 1
        if abs(p1_m-1) <= 0.05:
            p1_m = 1
            
        err_ind.append([p0_m,p1_m])
        
    for pp in err_ind:
        if pp[0]<-0.2 or pp[0]>1.2 or pp[1]<-0.2 or pp[1]>1.2:
            pp[0], pp[1] = 0.5, 0.5
        else:
            pp[0] = max(min(pp[0],1-kappa),kappa)
            pp[1] = max(min(pp[1],1-kappa),kappa)
    return err_ind, P0_est
    

In [ ]:
def sur_agg_1(preds_matrix, err_ind_as_a_list, P0_est, kappa=0.02): 
    
    num_q = int(len(preds_matrix)/1)
    num_w = len(preds_matrix[0])
    
    #for pp in err_ind_as_a_list:
    #    pp[0] = max(min(pp[0],1-kappa),kappa)
    #    pp[1] = max(min(pp[1],1-kappa),kappa)
        
    ###aggregating predictions using learned error rates.
    ###aggregator one
    L = [0 for i in range(num_w)]
    num_iter = 10 #number of samples to draw
    preds_mts = [0 for q in range(num_q)]

    for q in range(num_q):
        
        w_list = []
        preds_temp = preds_matrix[q]
        cnt_ww = 0
        
        for w in range(num_w):
            ind_ww = 1
            
            if (max(err_ind_as_a_list[w]) == kappa) or (sum(err_ind_as_a_list[w]) == 1):
                    ind_ww = -1
            
            if float(preds_temp[w]) != -1 and sum(err_ind_as_a_list[w]) <= 0.2 and ind_ww != -1 and cnt_ww <= 4:
                w_list.append(w)
                cnt_ww = cnt_ww + 1
        
        #print(len(w_list))
        L = [0 for i in range(len(w_list))]
        Prob_Y = 0
        #print len(w_list)
        for n in range(num_iter):
            prob_temp1 = 1
            prob_temp2 = 1
            cnt_iter = 0
            
            for w in range(len(w_list)):
                e0_temp = err_ind_as_a_list[w_list[w]][0] 
                e1_temp = err_ind_as_a_list[w_list[w]][1]
                #print e0_temp,e1_temp
                ind_ww = 1
                if (e0_temp+e1_temp == 1): #(max(e0_temp,e1_temp) == 0.1) or 
                    ind_ww = -1
                if min(e0_temp,e1_temp) > 0 and max(e0_temp,e1_temp) < 1 and ind_ww == 1:
                    toss_temp = uniform(0,1)
                    if toss_temp < float(preds_temp[w_list[w]]):
                        L[w] = 1
                        #prob_temp1 = prob_temp1*float(preds_temp[w_list[w]])
                        #prob_temp1 = prob_temp1*(1-e1_temp)
                        prob_temp2 = prob_temp2+np.log(e0_temp/(1-e1_temp))
                    else:
                        L[w] = 0
                        #prob_temp1 = prob_temp1*(1-float(preds_temp[w_list[w]]))
                        #prob_temp1 = prob_temp1*e1_temp
                        prob_temp2 = prob_temp2+np.log((1-e0_temp)/e1_temp)
            #print prob_temp1,prob_temp2
            #print prob_temp2
            Prob_Y = Prob_Y + prob_temp1*(1-P0_est)/((1-P0_est)+np.exp(prob_temp2)*P0_est)
        
        #print prob_temp1,Prob_Y
        
        preds_mts[q] = Prob_Y/num_iter
    return np.array(preds_mts)



In [ ]:
def sur_agg_2(preds_matrix, err_ind_as_a_list):
    num_q = int(len(preds_matrix)/1)
    num_w = len(preds_matrix[0])
    
    ###aggregator two
    L = [0 for i in range(num_w)]
    num_iter = 100 #number of samples to draw
    preds_mts = [0 for q in range(num_q)]
    for q in range(num_q):
        w_list = []
        preds_temp = preds_matrix[q]
        for w in range(num_w):
            ind_ww = 1
            if  (sum(err_ind_as_a_list[w]) == 1): #(max(err_ind[w]) == 0.1) or
                    ind_ww = -1
            
            if float(preds_temp[w]) != -1 and sum(err_ind_as_a_list[w]) <= 0.1 and ind_ww != -1:
                #the choice of threshold will also depend on how many forecasts are available. 
                w_list.append(w)
        
        weight_all = 0
        weight_pred = 0
        
        for ww in w_list:
            weight_temp = 1-err_ind_as_a_list[ww][0]-err_ind_as_a_list[ww][1]
            #print weight_temp
            #weight_temp = np.exp(1-err_ind[ww][0]-err_ind[ww][1])

            #weight_all = weight_all + weight_temp
            #weight_pred = weight_pred + float(preds_temp[ww])*weight_temp #+ (1-float(preds_temp[ww]))*(1-weight_temp)
            #if float(preds_temp[ww]) > err_ind[ww][0]:
            #if weight_temp != 0:
            bonus = 0.2
            #if (float(preds_temp[ww]) - err_ind[ww][0])/weight_temp >= 0 and (float(preds_temp[ww]) - err_ind[ww][0])/weight_temp <= 1: 
            weight_pred = weight_pred + max(0,min((float(preds_temp[ww]) - err_ind_as_a_list[ww][0])/weight_temp,1+bonus))#bonus!
            weight_all = weight_all + 1
            #weight_pred = weight_pred + float(preds_temp[ww])*weight_temp + (1-float(preds_temp[ww]))*(1-weight_temp)
            #else:
                #weight_pred = weight_pred + float(preds_temp[ww])
        
        if weight_all<=0: preds_mts[q] = -1
        else: preds_mts[q] = weight_pred/weight_all
        
    return np.array(preds_mts)